In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def cartesianToSpherical(points):
    """
    Convert a point from cartesian coordinates to
    generalized spherical coordinates.

    The input cartesian coordinates should follow the order:
        (x, y, ...)

    The output spherical coordinates will follow the order:
        (r, t_1, t_2, t_3, ..., p)

    where `r` is the radius, the angles `t_i` are bounded between
    `[0, pi]`, and the angle p is bounded between `[0, 2pi]`.

    Parameters
    ----------
    points : numpy.ndarray[d] or numpy.ndarray[N,d]
        One or multiple sets of points in cartesian coordinates.

    Returns
    -------
    cartesianPoints : numpy.ndarray[d] or numpy.ndarray[N,d]
        Output always matches the shape of the input `points`.

    """
    nPoints = np.shape(points)[0] if len(np.shape(points)) > 1 else 1
    dim = np.shape(points)[-1]

    if nPoints == 1 and len(np.shape(points)) == 1:
        arrPoints = np.array([points])
    else:
        arrPoints = np.array(points)

    # See note about this roll in sphericalToCartesian function.
    # It is to make sure the points can come in as (x,y,z...)
    if dim >= 3:
        arrPoints = np.roll(arrPoints, 1, axis=-1)

    sphericalPoints = np.zeros((nPoints, dim))
    # See the page on n-spheres for these equations:
    # https://en.wikipedia.org/wiki/N-sphere#Spherical_coordinates
    # Radius
    sphericalPoints[:,0] = np.sqrt(np.sum(arrPoints**2, axis=-1))
    for i in range(1, dim-1):
        sphericalPoints[:,i] = np.arctan2(np.sqrt(np.sum(arrPoints[:,i:]**2, axis=-1)), arrPoints[:,i-1])
    # The unique angle
    sphericalPoints[:,-1] = np.arctan2(arrPoints[:,-1], arrPoints[:,-2])

    # Remove extra dimensions if you only have a single point
    return sphericalPoints[0] if (nPoints == 1 and len(np.shape(points)) == 1) else sphericalPoints
   

def sphericalToCartesian(points):
    """
    Convert a point from generalized spherical coordinates to
    cartesian coordinates.

    The input spherical coordinates should follow the order:
        (r, t_1, t_2, t_3, ..., p)

    where `r` is the radius, the angles `t_i` are bounded between
    `[0, pi]`, and the angle p is bounded between `[0, 2pi]`.

    This transformation is consistent in 2D with:
        (r cos(p), r sin(p))
    and in 3D with:
        (r sin(t) cos(p), r sin(t) sin(p), r cos(t))
        
    Parameters
    ----------
    points : numpy.ndarray[d] or numpy.ndarray[N,d]
        One or multiple sets of points in spherical coordinates.

    Returns
    -------
    cartesianPoints : numpy.ndarray[d] or numpy.ndarray[N,d]
        Output always matches the shape of the input `points`.
    """
    nPoints = np.shape(points)[0] if len(np.shape(points)) > 1 else 1
    dim = np.shape(points)[-1]

    if nPoints == 1 and len(np.shape(points)) == 1:
        arrPoints = np.array([points])
    else:
        arrPoints = np.array(points)

    cartesianPoints = np.zeros((nPoints, dim))
    for i in range(dim-1):
        cartesianPoints[:,i] = arrPoints[:,0] * np.product(np.sin(arrPoints[:,1:i+1]), axis=-1) * np.cos(arrPoints[:,i+1])
    # The last one is different
    cartesianPoints[:,-1] = arrPoints[:,0] * np.product(np.sin(arrPoints[:,1:]), axis=-1)

    # For some reason, the calculation above shifts the order of
    # the cartesian components by one IF the dimension is greater than 3...
    # I really don't know why, but adding this line (and a complementary one
    # in sphericalToCartesian) makes sure that we always have the order
    # (x,y,z...)
    if dim >= 3:
        cartesianPoints = np.roll(cartesianPoints, -1, axis=-1)
    
    # Remove extra dimensions if you only have a single point
    return cartesianPoints[0] if (nPoints == 1 and len(np.shape(points)) == 1) else cartesianPoints


In [ ]:
%matplotlib widget
for _ in range(10):
    plt.close()

tArr = np.linspace(0, np.pi, 20)
pArr = np.ones(len(tArr))
rArr = np.ones(len(tArr))
sphericalPoints = np.array(list(zip(rArr, tArr, pArr)))

cartesianPoints = sphericalToCartesian(sphericalPoints)

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

ax.scatter(*cartesianPoints.T)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
plt.show()

In [ ]:
point = [1, np.pi/2, 0]
t = point[1]
p = point[2]

dir = np.array([np.sin(t)*np.cos(p),
                            np.sin(t)*np.sin(p),
                            np.cos(t)])

print(point)
print(dir)
print(sphericalToCartesian(point))
print(cartesianToSpherical(sphericalToCartesian(point)))

In [ ]:
point = [1, np.pi/2]

dir = np.array([np.cos(point[1]), np.sin(point[1])])

print(point)
print(dir)
print(sphericalToCartesian(point))
print(cartesianToSpherical(sphericalToCartesian(point)))

In [ ]:
for i in range(5):
    d = i+2
    point = np.random.uniform(1, 2, size=d)

    print(f'd = {d}')
    print(f'spherical: {point}')
    print(f'spherical -> cart: {sphericalToCartesian(point)}')
    print(f'spherical -> cart -> spherical: {cartesianToSpherical(sphericalToCartesian(point))}\n')

    print(f'cart: {point}')
    print(f'cart -> spherical: {cartesianToSpherical(point)}')
    print(f'cart -> spherical -> cart: {sphericalToCartesian(cartesianToSpherical(point))}\n\n')
